In [95]:
import torch
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import numpy as np

import pandas as pd
import gensim.downloader as api
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
import numpy as np

In [96]:
crawling_result = pd.read_csv('인공지능_문화체육관광부_클라썸_crawled.csv')

In [97]:
# crawling_result

In [98]:
query = '인공지능'
business_name = '클라썸'

In [99]:
crawling_result['checker'] = crawling_result.crawling_trg.str.find(business_name)

bussiness_news = crawling_result[crawling_result.checker > -1].copy().reset_index(drop=True)
bussiness_news['idx_original'] = range(len(bussiness_news))

org_news = crawling_result[crawling_result.checker == -1].copy().reset_index(drop=True)
org_news['idx_original'] = range(len(org_news))

business_news_contents = bussiness_news[['idx_original', 'crawling_trg', 'pubDate', 'title', 'content', 'link']]
org_news_contents = org_news[['idx_original', 'crawling_trg', 'pubDate', 'title', 'content', 'link']]

In [100]:
def make_bow(query, articles):
    name = articles.crawling_trg.unique()[0].split(query)[0].strip()

    articles = articles.content.apply(lambda x: x.split()).to_list().copy()
    articles = [[word if not (query in word) else query for word in article] for article in articles]
    articles = [[word if not (name in word) else name for word in article] for article in articles]

    dct = Dictionary(articles)  # fit dictionary
    bow_articles = [dct.doc2bow(article) for article in articles]  # convert corpus to BoW format
    bow_articles = [{k:v for k, v in bow_article} for bow_article in bow_articles]
    
    return name, dct, bow_articles

In [101]:
def id_by_tf_retrieve(query, name, dct, bow_articles, query_th=0, name_th=0):
    print('query_th:', query_th)
    print('name_th:', name_th)
    query_id = dct.token2id[query]
    query_tfs = []
    for bow_article in bow_articles:
        query_tfs.append(bow_article[query_id])
    query_tfs = np.array(query_tfs)

    name_id = dct.token2id[name]
    name_tfs = []
    for bow_article in bow_articles:
        name_tfs.append(bow_article[name_id])
    name_tfs = np.array(name_tfs)
    
    tf_pairs = [(q, n) for q, n in zip (query_tfs, name_tfs)]
    print('query_name_tf_pairs: ', tf_pairs)
    ids = {}
    for id, tf_pair in enumerate(tf_pairs):
        query_tf, name_tf = tf_pair[0], tf_pair[1]
        if query_tf >= query_th and name_tf >= name_th:
            ids[id] = query_tfs[id] + name_tfs[id]
    print('ids over thresholds(>=):', ids)
    if bool(ids) == False:
        return None
    
    doc_id = max(ids, key=ids.get)
    print('doc_id:', doc_id)

    return doc_id

In [102]:
def show_article(doc_id, articles):
    print(articles.iloc[doc_id].crawling_trg)
    print(articles.iloc[doc_id].title)
    print(articles.iloc[doc_id].content)

In [103]:
# for _ in range(3):
#     for i, row in business_news_contents.iterrows():
#         print(row.content)
#         print()

In [104]:
# business_news_contents

name, dct, bow_articles = make_bow(query, business_news_contents)
print(name)
doc_id = id_by_tf_retrieve(query, name, dct, bow_articles, query_th=1, name_th=3)
if doc_id == None:
    print(f'{name} 관련 기사 없음.')
    print()
else:
    show_article(doc_id, business_news_contents)
    print()

클라썸
query_th: 1
name_th: 3
query_name_tf_pairs:  [(1, 1), (1, 1), (1, 3), (1, 2), (1, 9), (1, 1), (1, 1), (1, 10)]
ids over thresholds(>=): {2: 4, 4: 10, 7: 11}
doc_id: 7
클라썸 인공지능
최유진·이채린 공동대표, 마케터·개발자로 만나 1시간 만에 창업 의기투합...
세계 6천개 대학·기업 '소통 플랫폼' 클라썸 최유진·이채린 공동대표 KAIST 선·후배 관계 "교육문제 풀려는 창업동기 닮아"교사·학생이 함께 쓰는 SNS삼성·LG도 사내 소통용 '각광'만난 지 1시간 만에 공동 창업을 결심하고 5년째 스타트업을 이끌어온 두 사람이 있다. 글로벌 에듀테크 스타트업으로 주목받고 있는 클라썸의 최유진(왼쪽), 이채린(오른쪽) 각자 대표다. 이들은 어떻게 의기투합하게 됐을까. 9일 서울 강남역 인근 사무실에서 만난 이 대표는 “최 대표는 열정적인 마케터이고 저는 논리적인 개발자여서 성향은 다른 편”이라면서도 “교육 문제를 풀고자 하는 창업 동기가 닮아 같은 길을 가게 됐다”고 설명했다. 돈을 벌려는 목적보다는 실질적으로 교육 문제를 해결하려는 목표가 더 절실했다는 것이다.클라썸은 회사명이자 이들이 개발한 ‘교육 소통 플랫폼’ 명칭이기도 하다. 클라썸은 수업 시 교사와 학생들이 질문과 답변을 올릴 수 있는 일종의 SNS다. 인공지능(AI) 조교인 도트(DOT) 기능도 갖추고 있다. 도트는 문맥과 관련 질문을 알아서 검색해주고, 중복 질문에 답변하는 역할을 한다.2018년 1월, 두 사람은 KAIST 창업원 선생님의 소개로 만났다. 이 대표는 KAIST 전산학부 2학년에 재학 중 학생과 교수들이 소통할 수 있는 온라인 플랫폼을 개발했다. 하지만 수익모델을 만들고, 이를 팔아줄 사람이 필요했다. KAIST 대학원생이었던 최 대표도 교육용 앱 창업을 준비 중이었다. 이 대표를 앱 제작을 맡아줄 ‘외주 사장님’이라고 생각한 그는 파워포인트 자료까

In [105]:
org_news_contents_splits = []
for org in org_news_contents.crawling_trg.unique():
    org_news_contents_split = org_news_contents[org_news_contents.crawling_trg == org].reset_index(drop=True).copy()
    org_news_contents_splits.append(org_news_contents_split)

In [106]:
for articles in org_news_contents_splits:
    name, dct, bow_articles = make_bow(query, articles)
    doc_id = id_by_tf_retrieve(query, name, dct, bow_articles, query_th=2, name_th=3)
    if doc_id == None:
        print(f'{name} 관련 기사 없음.')
        print()
        continue
    show_article(doc_id, articles)
    print()

query_th: 2
name_th: 3
query_name_tf_pairs:  [(3, 1), (1, 2), (1, 1), (9, 1), (3, 13), (1, 1), (6, 6), (2, 2), (1, 3), (1, 3), (1, 2), (5, 3), (2, 2), (1, 2), (3, 1), (5, 4), (3, 6), (2, 4), (2, 2), (3, 4), (2, 4), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 7), (1, 3), (1, 6), (1, 6), (1, 5), (1, 1)]
ids over thresholds(>=): {4: 16, 6: 12, 11: 8, 15: 9, 16: 9, 17: 6, 19: 7, 20: 6}
doc_id: 4
국립국어원 인공지능
[출근길 인터뷰] 고마워 한글…2022 한글 주간 개막
[앵커]올해로 576돌을 맞는 한글날을 맞아 '고마워 한글'이라는 주제로 오늘부터 일주일간 다채로운 한글 주간 행사가 열립니다. 국립국어원에서는 '세상을 잇는 한국어, 미래를 이끄는 한글'이라는 주제로 국제학술대회를 개최하는데요. 오늘 출근길 인터뷰에서는 장소원 국립국어원장을 만나봅니다. 박서휘 캐스터 전해주시죠.[캐스터] 출근길인터뷰입니다. 오늘은 장소원 국립국어원장을 만나 얘기 나눠보겠습니다. 안녕하세요.[장소원 국립국어원장]안녕하세요.[캐스터] 쉽고 편리한 한글이 우리에게 주는 고마움을 되새기고자 고마워 한글이라는 주제로 오늘부터 전국 곳곳에서 행사가 열리는데요. 어떤 의미를 담은 행사인지 소개해 주시죠.[장소원 국립국어원장]오늘부터 다음 주 월요일인 10월 10일까지 한글날을 기념하는 2022 한글주관 행사가 국립한글박물관을 비롯한 전국 각지에서 열립니다. 특히 올해는 쉽고 편리한 한글에 대한 고마움을 되새기고자 '고마워 한글'이라는 주제로 국립국어원, 국립한글박물관, 세종학당 재단 등의 기관과 여러 문화 단체들이 참여해서 전야제

In [297]:
# show_article(doc_id)